In [14]:
import requests
import json
import pandas as pd
import matplotlib.pyplot as plt

In [15]:
weather_url = "https://api.open-meteo.com/v1/forecast?latitude=39.7392&longitude=-104.9847&hourly=temperature_2m,precipitation,precipitation_probability,wind_speed_10m,wind_direction_10m&timezone=America%2FDenver&wind_speed_unit=mph&temperature_unit=fahrenheit&precipitation_unit=inch"

In [16]:
def get_json_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()  # Raise HTTPError for bad responses (4xx or 5xx)
        return response.json()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return None
    except json.JSONDecodeError:
        print("Failed to decode JSON")
        return None

In [ ]:
weather_json = get_json_from_url(weather_url)

weather_df = pd.DataFrame.from_dict(weather_json['hourly'])
weather_df.head()

,time,temperature_2m,precipitation,precipitation_probability,wind_speed_10m,wind_direction_10m
0,2025-05-14T00:00,64.8,0.0,0,2.3,107
1,2025-05-14T01:00,65.5,0.0,0,6.0,243
2,2025-05-14T02:00,59.7,0.0,0,5.9,252
3,2025-05-14T03:00,57.0,0.0,0,3.2,225
4,2025-05-14T04:00,54.2,0.0,0,4.6,67


In [18]:
weather_df['time'] = pd.to_datetime(weather_df['time'])
weather_df['day_name'] = weather_df['time'].dt.strftime('%a')
weather_df['hour'] = weather_df['time'].dt.hour


weather_df.loc[weather_df['hour'] < 25, 'day_part'] = 'night'
weather_df.loc[weather_df['hour'] < 19, 'day_part'] = 'after work'
weather_df.loc[weather_df['hour'] < 15, 'day_part'] = 'early afternoon'
weather_df.loc[weather_df['hour'] < 12, 'day_part'] = 'morning'
weather_df.loc[weather_df['hour'] < 9, 'day_part'] = 'night'

weather_df.head()

,time,temperature_2m,precipitation,precipitation_probability,wind_speed_10m,wind_direction_10m,day_name,hour,day_part
0,2025-05-14 00:00:00,64.8,0.0,0,2.3,107,Wed,0,night
1,2025-05-14 01:00:00,65.5,0.0,0,6.0,243,Wed,1,night
2,2025-05-14 02:00:00,59.7,0.0,0,5.9,252,Wed,2,night
3,2025-05-14 03:00:00,57.0,0.0,0,3.2,225,Wed,3,night
4,2025-05-14 04:00:00,54.2,0.0,0,4.6,67,Wed,4,night


In [19]:
day_part_df = weather_df.groupby(['day_name', 'day_part']).agg(
    day_start = ('time', 'min'),
    temp_high = ('temperature_2m', 'max'),
    temp_low = ('temperature_2m', 'min'),
    hourly_precip = ('precipitation', 'mean'),
    precip_prob_peak = ('precipitation_probability', 'max'),
    wind_speed_peak = ('wind_speed_10m', 'max'),
    wind_spead_avg = ('wind_speed_10m', 'mean'),
    wind_direction_avg = ('wind_direction_10m', 'mean')
).reset_index().sort_values('day_start').drop('day_start', axis=1)

In [21]:
after_work_df = day_part_df[day_part_df['day_part'] == 'after work']
after_work_df

,day_name,day_part,temp_high,temp_low,hourly_precip,precip_prob_peak,wind_speed_peak,wind_spead_avg,wind_direction_avg
24,Wed,after work,73.0,60.6,0.000,21,20.9,16.800,215.50
16,Thu,after work,68.5,66.7,0.000,5,17.6,14.775,307.25
0,Fri,after work,72.4,71.2,0.000,6,15.9,14.375,342.75
8,Sat,after work,71.2,66.1,0.001,17,12.9,9.550,238.75
12,Sun,after work,70.3,65.8,0.000,28,24.7,16.525,272.25
4,Mon,after work,73.0,65.4,0.000,52,27.9,18.425,350.75
20,Tue,after work,62.9,58.6,0.000,24,6.6,3.875,262.50


In [39]:
from PIL import Image, ImageDraw, ImageFont

x_size = 250
y_size = 122
out = Image.new("RGB", (x_size, y_size), (255, 255, 255))
fnt = ImageFont.truetype("/usr/share/fonts/liberation/LiberationMono-Regular.ttf", 12)
title = ImageFont.truetype("/usr/share/fonts/liberation/LiberationMono-Bold.ttf", 15)
d = ImageDraw.Draw(out)

n_days = 5

for i, row in enumerate(after_work_df.head(n_days).iterrows()):
    row = row[1]
    x_pos = i * x_size / n_days
    d.line(xy=[(x_pos, 0), (x_pos, y_size)], fill=(0,0,0), width=1)

    to_print = ['day_name', 'temp_high', 'temp_low', 'precip_prob_peak', 'wind_spead_avg']
    current_y_pos = 15
    for col in to_print:
        val = row[col]

        if col == 'day_name':
            d.text(xy=(x_pos + (x_size/n_days/2), current_y_pos), text=val, anchor='mt', font=title, fill=(0,0,0))
            current_y_pos += title.size + 5
        else:
            val = str(round(val, 2))
            d.text(xy=(x_pos + (x_size/n_days/2), current_y_pos), text=val, anchor='mt', font=fnt, fill=(0,0,0))
            current_y_pos += fnt.size + 2

out.save('test.png')

In [26]:
for i, row in enumerate(after_work_df.head(n_days).iterrows()):
    print(row[1]['day_name'])

Wed
Thu
Fri
Sat
Sun
Mon
Tue


In [30]:
row['temp_high']

73.0